In [1]:
import os
import requests
import pandas as pd
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from git import Repo

In [2]:
CHECKPOINT = "neuralsentry/starencoder-finetuned-class"

model = AutoModelForSequenceClassification.from_pretrained(CHECKPOINT)
tokenizer = AutoTokenizer.from_pretrained(CHECKPOINT)

In [ ]:
Repo.clone_from("https://github.com/redis/redis", "test/repos/redis")

<git.repo.base.Repo 'd:\\Projects\\neural-sentry\\test\\repos\\redis\\.git'>

In [3]:
repo = Repo("test/repos/redis")

In [6]:
TARGET_BUG_COMMITS = 50
buggy_commits = []

In [7]:
threshold = 0.9

for commit in repo.iter_commits():
    commit_msg = commit.message
    outputs = model(
        **tokenizer(commit_msg, return_tensors="pt", truncation=True, max_length=512)
    )
    logits = outputs.logits
    pred = logits.argmax().item()
    pred_softmax = logits.softmax(dim=1).tolist()[0]

    if pred_softmax[1] > threshold:
        buggy_commits.append(commit)
        print(f"[*] Predicted (bug): {pred}, {pred_softmax}")
        print(f"[+] Found {len(buggy_commits)} buggy commits")
    else:
        print(f"[*] Predicted (non-bug): {pred}, {pred_softmax}")
        print(
            f'[-] Found "{commit_msg[:50] + "..." if len(commit_msg) > 50 else commit_msg} non-buggy commit"'
        )

    if len(buggy_commits) == TARGET_BUG_COMMITS:
        break

[*] Predicted (bug): 1, [0.0035345929209142923, 0.9964654445648193]
[+] Found 1 buggy commits
[*] Predicted (non-bug): 0, [0.9954971075057983, 0.0045029218308627605]
[-] Found "Fix typos in comments (#12338)

 non-buggy commit"
[*] Predicted (bug): 1, [0.0035337882582098246, 0.9964661598205566]
[+] Found 2 buggy commits
[*] Predicted (bug): 1, [0.0017395206959918141, 0.9982604384422302]
[+] Found 3 buggy commits
[*] Predicted (bug): 1, [0.03995134308934212, 0.9600487351417542]
[+] Found 4 buggy commits
[*] Predicted (non-bug): 0, [0.9898627996444702, 0.010137242265045643]
[-] Found "Align RM_ReplyWithErrorFormat with RM_ReplyWithErr... non-buggy commit"
[*] Predicted (bug): 1, [0.0029292027465999126, 0.9970707893371582]
[+] Found 5 buggy commits
[*] Predicted (bug): 1, [0.006082089617848396, 0.9939179420471191]
[+] Found 6 buggy commits
[*] Predicted (bug): 1, [0.011138077825307846, 0.9888619780540466]
[+] Found 7 buggy commits
[*] Predicted (non-bug): 1, [0.36370930075645447, 0.636290

In [8]:
commit_msgs = [commit.message for commit in buggy_commits]
df = pd.DataFrame(commit_msgs, columns=["commit_msg"])

In [9]:
df.head(10)

,commit_msg
0,Modules: Unblock from within a timer coverage ...
1,Print strerror when bio initialization fails (...
2,Improve moduleBlockClient timeout overflow han...
3,Make nodename test more consistent (#12330)\n\...
4,Fix broken protocol when PUBLISH emits local p...
5,use embedded string object and more efficient ...
6,zrangeGenericCommand add check for negative of...
7,Sanitizer reported memory leak for '--invalid'...
8,Fix cluster human_nodename Getter data loss in...
9,Set Jemalloc --disable-cache-oblivious to redu...
